## 2.4 Send Data to Serial

- Install `pyserial` in Anaconda prompt
    - Open Anaconda Prompt
    - Open environment `BelajarOpenCV`
        ```bash
        conda activate BelajarOpenCV
    - install `pyserial`
        ```bash
        pip install pyserial

- Here's a basic Python script to send data to serial (UART) connection using the pyserial library.
    - Modify the `SERIAL_PORT` and `BAUD_RATE` according to your device in device manager.

In [ ]:
import serial
import time

In [ ]:
# Set the correct serial port and baud rate
SERIAL_PORT = "COM3"  # Change to your Arduino's port (e.g., "/dev/ttyUSB0" for Linux/macOS)
BAUD_RATE = 9600      # Must match the baud rate in your Arduino code

# Open serial connection
try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    time.sleep(2)  # Give the connection a moment to initialize
    print(f"Connected to {SERIAL_PORT} at {BAUD_RATE} baud.")

    while True:
        data = input("Enter data to send: ")  # Get user input
        ser.write(data.encode())  # Send data as bytes
        print(f"Sent: {data}")

        time.sleep(0.5)  # Small delay to avoid overwhelming the Arduino

except serial.SerialException as e:
    print(f"Error: {e}")

finally:
    ser.close()
    print("Serial connection closed.")

- Example Arduino Code
- Upload this to your Arduino to receive data from Python.
    ```cpp
    void setup() {
        Serial.begin(9600);
    }

    void loop() {
        if (Serial.available()) {
            String received = Serial.readStringUntil('\n');
            Serial.print("Received: ");
            Serial.println(received);
        }
    }

#### 2.3.2 Send Detected Object Centroid and Orientation to Serial

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

print("OpenCV version: " + cv2.__version__)

In [ ]:
# Serial communication setup (change COM port as needed)
SERIAL_PORT = "COM3"  # Change for Linux/macOS, e.g., "/dev/ttyUSB0"
BAUD_RATE = 9600

try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    time.sleep(2)  # Give some time for the connection to initialize
    print(f"Connected to {SERIAL_PORT} at {BAUD_RATE} baud.")
except serial.SerialException as e:
    print(f"Serial connection error: {e}")
    ser = None

# Load and convert the image to grayscale
image = cv2.imread("hands.jpg")  # Change to "blocks.jpg" if needed
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Step 1: Apply Binary Thresholding
_, binary = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY)  # Adjust threshold if needed

# Step 2: Apply Canny Edge Detection
edges = cv2.Canny(binary, 50, 200)

# Step 3: Find contours
contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Step 4: Process each detected contour
output = image.copy()
num_objects = 0  # Counter for detected objects

for contour in contours:
    if cv2.contourArea(contour) < 50:  # Skip small contours
        continue

    num_objects += 1

    # Convex hull to smooth contour shape
    contour = cv2.convexHull(contour)

    # Compute centroid
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        cv2.circle(output, (cx, cy), 3, (0, 0, 255), -1)  # Draw centroid

    # PCA for orientation detection
    data_points = np.array(contour.squeeze(), dtype=np.float64)
    mean, eigenvectors = cv2.PCACompute(data_points, mean=None)
    center = tuple(mean[0].astype(int))

    # Compute orientation angle
    angle_rad_original = np.arctan2(eigenvectors[0, 1], eigenvectors[0, 0])  
    angle_rad = (angle_rad_original - np.pi) % (-1 * np.pi)
    angle_rad = abs(angle_rad)
    angle_deg = int(np.rad2deg(angle_rad))  # Convert radians to degrees

    # Draw orientation lines
    length = 50
    cv2.line(output, center, (center[0] + length, center[1]), (0, 0, 0), 2)  # Reference line
    end_x = int(center[0] + length * np.cos(angle_rad))
    end_y = int(center[1] - length * np.sin(angle_rad))
    cv2.line(output, center, (end_x, end_y), (0, 0, 255), 2)  # Orientation line

    # Draw angle value
    text_position = (center[0] - 7, center[1] - 7)
    cv2.putText(output, f"{angle_deg}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)

    # Send centroid and angle data to Arduino via Serial
    if ser:
        serial_data = f"{cx},{cy},{angle_deg}\n"
        ser.write(serial_data.encode())
        print(f"Sent: {serial_data.strip()}")

# Display results
plt.figure(figsize=(15, 5))
plt.imshow(output[:, :, ::-1])  # Convert BGR to RGB
plt.axis("off")
plt.title(f"Detected Objects: {num_objects}")
plt.show()

# Close serial connection
if ser:
    ser.close()
